In [1]:
%%javascript
$.getScript('http://asimjalis.github.io/ipyn-ext/js/ipyn-present.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Data storage on the batch layer</h1>
<div id="toc"></div>

![The batch layer must structure large, continually growing datasets in a manner that supports low maintenance as well as efficient creation of the batch views](04fig01_alt.jpg)
* Determine the requirements for storing the master dataset
* See why distributed filesystems are a natural fit for storing a master dataset
* See how the batch layer storage for the SuperWebAnalytics.com project maps to
distributed filesystems

Storage requirements for the master dataset
------------------------------------------------------
<details>
  <summary>A checklist of storage requirements for the master dataset</summary>
<table>
<tr><th>Operation</th>
    <th>Requisite</th>
    <th>Discussion</th></tr>
<tr><td rowspan=2>Write</td>
    <td>Efficient appends of new data</td>
    <td>The only write operation is to add new pieces of data, so it must be easy and efficient to append a new set of data objects to the master dataset.</td></tr>
<tr><td>Scalable storage</td>
    <td>The batch layer stores the complete dataset—potentially terabytes or petabytes of data. It must therefore be easy to scale the storage as your dataset grows.</td></tr>
<tr><td>Read</td>
    <td>Support for parallel processing</td>
    <td>Constructing the batch views requires computing functions on the entire master dataset. The batch storage must consequently support parallel processing to handle large amounts of data in a scalable manner.</td></tr>
<tr><td rowspan=2>Both</td>
    <td>Tunable storage and processing costs</td>
    <td>Storage costs money. You may choose to compress your data to help minimize your expenses, but decompressing your data during computations can affect performance. The batch layer should give you the flexibility to decide how to store and compress your data to suit your specific needs.</td></tr>
<tr><td>Enforceable immutability</td>
    <td>It’s critical that you’re able to enforce the immutability property on your master dataset. Of course, computers by their very nature are mutable, so there will always be a way to mutate the data you’re storing. The best you can do is put checks in place to disallow mutable operations. These checks should prevent bugs or other random errors from trampling over existing data.</td></tr>
</table>
</details>

<!--redundant
Choosing a storage solution for the batch layer
------------------------------------------------------
### Using a key/value store for the master dataset
### Distributed filesystems

How distributed filesystems work
------------------------------------------------------
### Files are chunked into blocks, which are dispersed to datanodes in the cluster.
![Files are chunked into blocks, which are dispersed to datanodes in the cluster.](../Big_Data_files/04fig02_alt.jpg)
### Clients communicate with the namenode to determine which datanodes hold the blocks for the desired file.
![Clients communicate with the namenode to determine which datanodes hold the blocks for the desired file.](../Big_Data_files/04fig03_alt.jpg)
* Files are spread across multiple machines for scalability and also to enable parallel processing.
* File blocks are replicated across multiple nodes for fault tolerance.
-->

Storing a master dataset with a distributed filesystem
------------------------------------------------------
### Spreading the master dataset throughout many files
![Spreading the master dataset throughout many files](04fig04_alt.jpg)
### Appending to the master dataset by uploading a new file with new data records
![Appending to the master dataset by uploading a new file with new data records](04fig05_alt.jpg)

<table>
<caption>How distributed filesystems meet the storage requirement checklist</caption>
<tr><th>Operation</th>
    <th>Requisite</th>
    <th>Discussion</th></tr>
<tr><td rowspan=2>Write</td>
    <td>Efficient appends of new data</td>
    <td>Appending new data is as simple as adding a new file to the folder containing the master dataset.</td></tr>
<tr><td>Scalable storage</td>
    <td>Distributed filesystems evenly distribute the storage across a cluster of machines. You increase storage space and I/O throughput by adding more machines.</td></tr>
<tr><td>Read</td>
    <td>Support for parallel processing</td>
    <td>Distributed filesystems spread all data across many machines, making it possible to parallelize the processing across many machines. Distributed filesystems typically integrate with computation frameworks like MapReduce to make that processing easy to do (discussed in upcoming lessons)</td></tr>
<tr><td rowspan=2>Both</td>
    <td>Tunable storage and processing costs</td>
    <td>Just like regular filesystems, you have full control over how you store your data units within the files. You choose the file format for your data as well as the level of compression. You’re free to do individual record compres- sion, block-level compression, or neither.</td></tr>
<tr><td>Enforceable immutability</td>
    <td>Distributed filesystems typically have the same permissions systems you’re used to using in regular filesystems. To enforce immutability, you can dis- able the ability to modify or delete files in the master dataset folder for the user with which your application runs. This redundant check will protect your previously existing data against bugs or other human mistakes.</td></tr>
</table>

Vertical partitioning
------------------------------------------------------
### A vertical partitioning scheme for login data. 
![vertical partitioning scheme for login data](04fig06_alt.jpg)
By sorting information for each date in separate folders, a function can select only the folders containing data relevant to its computation.

Low-level nature of distributed filesystems
------------------------------------------------------
### An example of a folder of data you may want to append to a master dataset. 
![It’s possible for filenames to overlap.](04fig07.jpg)

The most obvious thing to try is something like
```bash
foreach file : "/new-data"    # Iterate over all files in /new-data
     mv file "/master/"       # Move the file into the /master folder
```

### If the target dataset is vertically partitioned, appending data to it is not as simple
![If the target dataset is vertically partitioned, appending data to it is not as simple as just adding files to the dataset folder.](04fig08_alt.jpg)

The graph schema for SuperWebAnalytics.com
------------------------------------------------------
![The graph schema for SuperWebAnalytics.com](04fig09_alt.jpg)

------------------------------------------------------
Lab
===============================
The unions within a graph schema provide a natural vertical partitioning scheme for a dataset.

    /data/
          person_property/
                          full_name/
                          gender/
                          location/
                          age/
          page_property/
                        page_views/
          equiv/
          page_view/
          page_link/

Write a batch job to vertically partition avro data according to the above scheme.  
*i.e.* Edges are partitioned according type. Properties are partitioned according to subtype.  
1. It is important that your solution scale horizontally. Writing a Python script load the data into memory and write it into different local files is easy. Writing a script that will leverage Hadoop or Spark's distributed architecture is a little more challenging.
2. Don't forget about the small-files problem. 
<!--3. Optional: The partitioning step is a good opportunity for deduping. Build that into your batch job as well.-->



***Hints*:**  
* MapReduce, whether implemented in Hadoop or in Spark, can only write to a single output directory at a time. As a result, it will be necessary to iterate over the dataset with filters in order to partition the data into different folders. For this it is recommended that you cache the data in an RDD.
* Spark's implementation of Avro is incomplete. It is therefore recommended that you use a different Avro library (*e.g.* [fastavro](https://pypi.python.org/pypi/fastavro/)). Libraries like this require a file object. If you are using Python, you may use [StringIO](https://docs.python.org/2/library/stringio.html#module-cStringIO) to solve this problem.
* Warning, be careful when dynamically generating filters. `lambda` in Python passes variables by reference, not by value, so you might find that you appear to be filtering on the same thing as you iterate through partitions. `def` saves a copy of the variable in the function.  
*e.g.* 
```python
rdd0 = sc.parallelize(range(10))
rdds = [rdd0.filter(lambda x: x%2 == i) for i in range(2)]
rdds[1].first() == rdds[0].first()
```
but
```python
def mod2_equals(i): return lambda x: x%2 == i
rdds = [rdd0.filter(mod2_equals(i)) for i in range(2)]
rdds[1].first() != rdds[0].first()
```